# Unsupervised Class Identification


* **Products used:** 
[gm_s2_annual](https://explorer.digitalearth.africa/gm_s2_annual), 
[gm_s2_semiannual](https://explorer.digitalearth.africa/gm_s2_semiannual), 
[fc_ls_summary_annual](https://explorer.digitalearth.africa/fc_ls_summary_annual),


## Background

## Description

The goal in this set of notebooks is to investigate whether unsupervised learning can be used identify individual crop-types in Niger.


---

## Getting Started

To run this analysis, run all the cells in the notebook, starting with the "**Load packages**" cell.

### Load packages

In [1]:
import os
import pickle

import datacube
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rioxarray
import xarray as xr
from datacube.utils import geometry
from datacube.utils.geometry import CRS, Geometry
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.classification import sklearn_flatten, sklearn_unflatten
from deafrica_tools.dask import create_local_dask_cluster
from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import display_map, rgb
from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.temporal import temporal_statistics, xr_phenology
from sklearn.cluster import DBSCAN, KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

### Set up a Dask cluster

Dask can be used to better manage memory use and conduct the analysis in parallel. 
For an introduction to using Dask with Digital Earth Africa, see the [Dask notebook](../Beginners_guide/08_Parallel_processing_with_dask.ipynb).

>**Note**: We recommend opening the Dask processing window to view the different computations that are being executed; to do this, see the *Dask dashboard in DE Africa* section of the [Dask notebook](../Beginners_guide/08_Parallel_processing_with_dask.ipynb).

To use Dask, set up the local computing cluster using the cell below.

In [2]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:35747 Dashboard: /user/victoria@kartoza.com/proxy/8787/status,Cluster Workers: 1 Cores: 4 Memory: 28.14 GB


### Analysis parameters

The following cell sets important parameters for the analysis:

* `vector_file`: file path for the shapefile containing the points of interest
* `time_01` : This is the time period of interest to load data for.
* `resolution` : The spatial resolution, in metres, to resample the satellite data to. 
* `output_crs` : The coordinate reference system that the loaded data is to be reprojected to.
* `dask_chunks`: the size of the dask chunks, dask breaks data into manageable chunks that can be easily stored in memory, e.g. dict(x=1000,y=1000)
* `output_dir` : The directory in which to store results from the analysis.
* `no_of_clusters` : The number of clusters to form as well as the number of centroids generated in the K-Means clustering.

**If running the notebook for the first time**, keep the default settings below.
This will demonstrate how the analysis works and provide meaningful results.

In [3]:
vector_file_1 = "../data/boundary_data/donnees_commune_kiota.geojson"
vector_file_2 = "../data/boundary_data/donnees_commune_koygolo.geojson"

# Time period of interest.
time_01 = "2021"

# Resolution and CRS to reproject data to.
resolution = (-20, 20)
output_crs = "EPSG:6933"

# Size of dask chunks.
dask_chunks = {"time": 1, "x": 500, "y": 500}

# Create the output directory to store the results.
output_dir = "results"
os.makedirs(output_dir, exist_ok=True)

# Specify the number of clusters to form as well as the number of centroids to generate in the K-Means Clustering.
no_of_clusters = 15

### Connect to the datacube

Connect to the datacube so we can access DE Africa data. 
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [4]:
dc = datacube.Datacube(app="unsupervised_class_identification")

### Load the shapefile

This shapefile contains the points of interest.

In [5]:
# Read in the vector_file.
gdf_1 = gpd.read_file(vector_file_1)
# Drop unnecessary columns from the GeoDataFrame.
gdf_1 = gdf_1.drop(
    columns=[
        "VILLAGE",
        "LATITUDE",
        "LONGITUD",
        "ANNEE",
        "LATDD",
        "LATMIN",
        "LAT",
        "LONDD",
        "LONMIN",
        "LON",
    ]
)
# Rename the columns.
gdf_1.rename(
    columns={"DEPARTEMEN": "Departemen", "COMMUNE": "Commune", "CULTURES": "Cultures"},
    inplace=True,
)

In [6]:
# Read in the vector_file.
gdf_2 = gpd.read_file(vector_file_2)
# Drop unnecessary columns from the GeoDataFrame.
gdf_2 = gdf_2.drop(
    columns=[
        "village",
        "coodos__go",
        "Latitude",
        "Field6",
        "Field7",
        "Longitude",
        "Field9",
        "Field10",
        "campagne_a",
        "Lat",
        "Lon",
    ]
)
# Rename the columns.
gdf_2.rename(columns={"Cultures_r": "Cultures"}, inplace=True)

In [7]:
# Merge the two GeoDataframes
gdf = pd.concat(objs=[gdf_1, gdf_2], axis=0, ignore_index=True)
# Add a unique id to gdf.
gdf["id"] = range(0, len(gdf))
gdf.explore()

## Defining feature layers

To create the desired feature layers, we pass instructions to `collect_training_data` through the `feature_func` parameter.

* `feature_func`: A function for generating feature layers that is applied to the data within the bounds of the input geometry. The `feature_func` must accept a `dc_query` dictionary, and return a single `xarray.Dataset` or `xarray.DataArray` containing 2D coordinates (i.e x, y - no time dimension). e.g.

          def feature_function(query):
              dc = datacube.Datacube(app='feature_layers')
              ds = dc.load(**query)
              ds = ds.mean('time')
              return ds

Below, we will define a more complicated feature layer function than the brief example shown above. 

In [8]:
# Define the `feature_layers` functions to load features.
def feature_layers(query):

    # Connnect to datacube.
    dc = datacube.Datacube(app="unsupervised_class_identification")

    # ----------------- Sentinel-2 Annual Geomedian -----------------

    # Sentinel-2 Annual and Bi-annual GeoMAD measurements.
    s2_measurements = [
        "blue",
        "green",
        "red",
        "nir",
        "swir_1",
        "swir_2",
        "red_edge_1",
        "red_edge_2",
        "red_edge_3",
        "BCMAD",
        "EMAD",
        "SMAD",
    ]

    # Load Sentinel-2 Annual Geomedian.
    ds_s2_annual_geomad = dc.load(
        product="gm_s2_annual", measurements=s2_measurements, **query
    )

    # Calculate some band indices.
    ds_s2_annual_geomad = calculate_indices(
        ds_s2_annual_geomad,
        index=["NDVI", "LAI", "SAVI", "MSAVI", "MNDWI"],
        drop=False,
        collection="s2",
    ).squeeze()

    # ----------------- Sentinel-2 Bi-Annual Geomedian -----------------

    # Load Sentinel-2 Bi-Annual Geomedian.
    ds_s2_semiannual_geomad = dc.load(
        product="gm_s2_semiannual",
        measurements=s2_measurements,
        **query,
    )

    # Split into two seperate datasets Jan-Jun and Jul-Dec.
    ds_janjun_geomad = (
        ds_s2_semiannual_geomad.isel(time=0)
        .reset_coords()
        .drop_vars(["time", "spatial_ref"])
    )
    ds_juldec_geomad = (
        ds_s2_semiannual_geomad.isel(time=1)
        .reset_coords()
        .drop_vars(["time", "spatial_ref"])
    )

    # Rename the dataset variables.
    janjun_rename_dict = {key: f"janjun_{key}" for key in list(ds_janjun_geomad.keys())}
    ds_janjun_geomad = ds_janjun_geomad.rename(name_dict=janjun_rename_dict)

    juldec_rename_dict = {key: f"juldec_{key}" for key in list(ds_juldec_geomad.keys())}
    ds_juldec_geomad = ds_juldec_geomad.rename(name_dict=juldec_rename_dict)

    # ----------------- S1 RADAR -----------------

    # Load Sentinel-1 data.
    #     ds_S1 = dc.load(
    #         product="s1_rtc",
    #         measurements=["vv", "vh", "mask"],
    #         group_by="solar_day",
    #         sat_orbit_state="ascending",
    #         **query,
    #     )

    # Get the temporal mean (mean over time) for the VH and VV polarizations.
    #     mean_vv = ds_S1.vv.where(ds_S1.mask == 1).mean(dim="time")
    #     mean_vh = ds_S1.vh.where(ds_S1.mask == 1).mean(dim="time")

    # Merge the temporal mean xarray.DataArrays into one xarray.Dataset.
    #     ds_S1_mean = xr.merge([mean_vv, mean_vh])

    # ----------------- Fractional Cover Annual Summaries -----------------

    # Fractional Cover Annual Summary  measurements.
    fc_measurements = [
        # "pv_pc_10",
        # "pv_pc_50",
        "pv_pc_90",
        # "npv_pc_10",
        # "npv_pc_50",
        "npv_pc_90",
        # "bs_pc_10",
        # "bs_pc_50",
        "bs_pc_90",
    ]

    # Load the Fractional Cover Annual Summary.
    ds_annual_fc = dc.load(
        product="fc_ls_summary_annual",
        measurements=fc_measurements,
        **query,
    )

    # ----------------- Temporal Statistics -----------------
    # Calculate these on NDVI over the year

    #     # Set parameters
    #     veg_index = "NDVI"
    #     resample_period = "2W"
    #     window = 4

    #     # Load S2 for query
    #     ds_S2 = load_ard(
    #         dc=dc,
    #         products=["s2_l2a"],
    #         measurements=["red", "nir"],
    #         mask_filters=[("opening", 3), ("dilation", 3)],
    #         **query,
    #         verbose=False,
    #     )

    #     # Calculate NDVI
    #     ds_S2 = calculate_indices(ds_S2, index=veg_index, collection="s2")

    #     # Smooth the NDVI over time
    #     ds_smooth = (
    #         ds_S2[veg_index]
    #         .resample(time=resample_period)
    #         .median()
    #         .rolling(time=window, min_periods=1)
    #         .mean()
    #     )

    #     statistics = [
    #         "discordance",
    #         #"f_mean",
    #         #"median_change",
    #         #"abs_change",
    #         "complexity",
    #         #"central_diff",
    #     ]

    #     ds_stats = temporal_statistics(ds_smooth, statistics)

    # ----------------- MERGE -----------------

    # Merge the indiviual datasets into one.
    ds_final = xr.merge(
        [
            ds_s2_annual_geomad,
            ds_janjun_geomad,
            ds_juldec_geomad,
            ds_annual_fc,
        ]  # ds_S1_mean
    )

    return ds_final.squeeze(dim="time", drop=True)

## Load data using the feature layers function

In [9]:
# Function to get the feature layers for a point.
def get_point_feature_layers(geom):
    # Generate a datacube query object.
    query = {
        "geopolygon": geom,
        "time": time_01,
        "resolution": resolution,
        "output_crs": output_crs,
        "dask_chunks": dask_chunks,
    }
    # Call the feature_layers function on the query to load the data.
    data = feature_layers(query)
    return data

In [10]:
%%time
# Get the feature layers for each point in the `gdf` GeoDataFrame.
id_list = gdf["id"].to_list()

# Define list to store the data for each point.
data_list = []

for ID in id_list:
    # print(f"Feature {ID:02d}/{len(id_list)}", end='\r')
    # Transform the point into a geometry object.
    geom = geometry.Geometry(geom=gdf.iloc[ID].geometry, crs=gdf.crs)
    # Get the NDVI profile for the point.
    point_data = get_point_feature_layers(geom).load()
    data_list.append(point_data)

CPLReleaseMutex: Error = 1 (Operation not permitted)


CPU times: user 28.6 s, sys: 945 ms, total: 29.5 s
Wall time: 59 s


In [11]:
# Since all the point datasets in data_list have the same variables.
# Get the list of variables from the first dataset on the list.
variables = list(data_list[0].keys())

# From each point dataset in data_list get the values for each variable.
data_vars = {}
for variable in variables:
    variable_values = [point_data[variable].data.item() for point_data in data_list]
    data_vars[variable] = variable_values

In [12]:
# Get the feature layers as a pandas DataFrame.
data_df = pd.DataFrame(data=data_vars)
data_df.head()

,blue,green,red,nir,swir_1,swir_2,red_edge_1,red_edge_2,red_edge_3,BCMAD,...,juldec_swir_2,juldec_red_edge_1,juldec_red_edge_2,juldec_red_edge_3,juldec_BCMAD,juldec_EMAD,juldec_SMAD,pv_pc_90,npv_pc_90,bs_pc_90
0,1131,1826,2682,3920,4574,3789,2903,3364,3626,0.050755,...,3695,2716,3262,3538,0.034209,774.552002,0.000827,38,42,63
1,979,1537,2189,3621,5110,4286,2808,3245,3535,0.055128,...,3880,2395,2928,3261,0.046550,984.015442,0.001648,33,41,73
2,1233,2025,3205,4323,6041,5442,3688,3935,4183,0.018953,...,5398,3569,3884,4152,0.019566,528.851685,0.000342,19,18,92
3,1213,1875,2859,4142,5281,4439,3260,3666,3946,0.037131,...,3991,2873,3405,3724,0.038483,861.882446,0.000919,34,33,77
4,1087,1793,2972,4181,5363,4504,3438,3834,4068,0.043657,...,3805,2883,3421,3705,0.053983,1160.647095,0.000996,27,45,78


## Classification using K-means Clustering

### Run Standard Scaler for all parameters

In [13]:
# Convert the `data_df` DataFrame to a matrix.
model_input = data_df.values

In [14]:
%%time
# Standardize the model_input (numpy array) by removing the mean and scaling to unit variance.
scaler = StandardScaler()
scaler.fit(model_input)
model_input = scaler.transform(model_input)

CPU times: user 1.16 ms, sys: 0 ns, total: 1.16 ms
Wall time: 919 µs


In [15]:
# Check for np.nan values in the scaled dataset.
np.unique(np.isnan(model_input))

array([False])

### Run K-Means Clustering

Perform k-means clustering on the standardized `model_input` numpy array.

In [16]:
%%time
# Set up the kmeans classification by specifying the number of clusters.
kmeans = KMeans(n_clusters=no_of_clusters, random_state=42)

# Perform Kmeans clustering on the 'model_input'.
# Begin iteratively computing the position of the clusters.
kmeans.fit(model_input)

# Use the sklearn kmeans `.predict` method to assign all the points of the `model_input` to a unique cluster.
predicted = kmeans.predict(model_input)
predicted

CPU times: user 330 ms, sys: 99.5 ms, total: 429 ms
Wall time: 135 ms


array([14,  9,  6,  0,  0,  0,  1,  3,  3, 11, 10, 13,  4,  1, 11, 11, 11,
        7,  2, 13,  0,  1,  8,  1,  5,  8,  2,  1,  1,  1,  1,  6,  2,  2,
       12,  6,  7, 12, 12,  2, 12, 12,  8,  8,  8,  8,  8,  8,  2,  6,  6,
        2,  8,  8, 10,  1,  1, 10, 10,  0,  1,  4,  4,  1,  1, 10,  0,  4,
        0, 10,  1,  4, 10,  4,  1,  6, 14, 10, 10,  1,  5,  5, 10],
      dtype=int32)

In [17]:
# Assign the predicted classes/clusters to the `gdf` GeoDataFrame.
gdf["predicted"] = predicted
gdf.head()

,Departemen,Commune,Cultures,geometry,id,predicted
0,Boboye,Kiota,mil et niébé,POINT (2.96908 13.33147),0,14
1,Boboye,Kiota,mil et niébé,POINT (2.93870 13.25270),1,9
2,Boboye,Kiota,mil et niébé,POINT (2.94553 13.29112),2,6
3,Boboye,Kiota,mil et niébé,POINT (2.97375 13.28498),3,0
4,Boboye,Kiota,mil et niébé,POINT (2.98262 13.29145),4,0


## Export the results

In [18]:
# Export the results as a geojson file.
gdf.to_file(
    f"{output_dir}/kmeans_{no_of_clusters}_classes_predicted_{time_01}.geojson",
    driver="GeoJSON",
)

## Recommended next steps

To continue working through the notebooks in this workflow, go to the next notebook `02_group_classes_using_NDVI.ipynb`.

1. **Unsupervised Class Identification (this notebook)**
2. [Group Classes Using NDVI](02_group_classes_using_NDVI.ipynb.ipynb)
3. [Reclassify Unsupervised NDVI](03_reclassify_unsupervised_map_NDVI_classes.ipynb)

---

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to repoart an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:**

In [19]:
print(datacube.__version__)

1.8.6


**Last Tested:**

In [20]:
from datetime import datetime

datetime.today().strftime("%Y-%m-%d")

'2022-06-30'